In [12]:
import string
import time
import random

from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.wait import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from fake_useragent import UserAgent

In [18]:
NASDAQ_URL = 'https://www.nasdaq.com/market-activity/stocks/screener'
companies = {}

ua = UserAgent()
user_agent = ua.random
options = webdriver.ChromeOptions()
options.add_argument(f'--user-agent={user_agent}')

prefs={"download.default_directory":"/Users/liammaguire/Desktop/"}
options.add_experimental_option("prefs",prefs)

#options.add_argument("--headless=new")

driver = webdriver.Chrome(options=options)

# Download CSVs for all NASDAQ & NYSE listings

# Load NASDAQ website
driver.get(NASDAQ_URL)
WebDriverWait(driver, 60).until(
    EC.presence_of_element_located((By.XPATH, '//*[@id="filterModal"]/div/div[3]/button[1]'))
)

def apply_and_download():
    time.sleep(5)
    apply_button = driver.find_element(By.XPATH, '//*[@id="filterModal"]/div/div[3]/button[1]')
    apply_button.click()
    time.sleep(5)
    download_button = driver.find_element(By.XPATH, '/html/body/div[2]/div/main/div[2]/article/div[3]/div[1]/div/div/div[3]/div[2]/div[2]/div/button')
    download_button.click()

# Apply changes and download NASDAQ csv
nasdaq_button = driver.find_element(By.XPATH, '//*[@id="radioItemNASDAQ"]')
nasdaq_button.click()
apply_and_download()
 
# Apply changes and download NYSE csv
nyse_button = driver.find_element(By.XPATH, '//*[@id="filterModal"]/div/div[2]/div[1]/div/div[2]/label/span[1]')
nyse_button.click()
apply_and_download()
driver.quit()